In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Polycystic_Kidney_Disease/GSE74453'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of a novel risk factor for intracranial aneurysms in ADPKD using iPSC models"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['subject/sample source id: TIG114 4F1', 'subject/sample source id: TIG118 4F1', 'subject/sample source id: TIG119 4F1', 'subject/sample source id: TIG121 4F4', 'subject/sample source id: TIG975E4', 'subject/sample source id: 585A1', 'subject/sample source id: 585B1', 'subject/sample source id: TIG107 4F1', 'subject/sample source id: TIG120 4F1', 'subject/sample source id: TIG120 3F7', 'subject/sample source id: TIG975E2', 'subject/sample source id: CiRA00004', 'subject/sample source id: CiRA00005', 'subject/sample source id: CiRA00006', 'subject/sample source id: CiRA00007', 'subject/sample source id: CiRA00008', 'subject/sample source id: CiRA00009', 'subject/sample source id: CiRA00010']

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Assuming the dataset likely contains gene expression data

# Variable Availability and Data Type Conversion

# 2.1 Data Availability
sample_char_dict = {
    0: ['subject/sample source id: CiRA00004', 'subject/sample source id: CiRA00005', 'subject/sample source id: CiRA00006', 'subject/sample source id: CiRA00007', 'subject/sample source id: CiRA00008', 'subject/sample source id: CiRA00009', 'subject/sample source id: CiRA00010'],
    1: ['subject status: autosomal dominant polycystic kidney disease (ADPKD) patient'],
    2: ['gender: Female', 'gender: Male'],
    3: ['cell type: iPSC derived endothelial cells', 'cell type: iPSC derived vascular smooth muscle cells']
}

# Checking availability and appropriate row numbers for each variable
if len(set(sample_char_dict[1])) > 1:
    trait_row = 1
if len(set(sample_char_dict[2])) > 1:
    gender_row = 2

# ADPKD (Polycystic Kidney Disease) is typically binary
def convert_trait(value):
    if 'ADPKD' in value:
        return 1
    return None

# Age is not available in the provided sample characteristics
# Implementing gender conversion: Female to 0, Male to 1
def convert_gender(value):
    gender = value.split(': ')[1].strip()
    if gender == 'Female':
        return 0
    elif gender == 'Male':
        return 1
    return None

# Save Metadata
save_cohort_info('GSE74453', './preprocessed/Polycystic_Kidney_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Polycystic_Kidney_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Polycystic_Kidney_Disease/trait_data/GSE74453.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Polycystic_Kidney_Disease/cohort_info.json
